# Jupyter IPykernel file code

Set all relative paths and import libraries.

In [ ]:
import sys
from pathlib import Path

# Manage file imports with Jupyter and iPykernel
notebook_dir = Path().resolve()
project_dir = Path().resolve().parent
sys.path.insert(0, str(project_dir))
sys.path.insert(0, str(notebook_dir))

# Add src/ to sys.path
src_path = project_dir / "src"
if src_path.exists():
    sys.path.append(str(src_path))

from datetime import datetime

# Initialize Variables
- Set the LLM library.
- Load the prompts from the YAML file.


In [ ]:
# Initialize variables
today = datetime.today()
llm = LLM()

# Define input and output directories
prompt_file_path = '../prompts'
response_file_path = '../outputs/responses'
presentation_file_path = '../outputs/presentations'
pdf_file_path = '../outputs/pdfs'

### Prompt Chain 1. Basic news research

In [ ]:
# Import the structured response schema
from projects.monday_briefing.schema import NewsInsightList, NewsReviewList, SummaryResponseList

# Load all the prompts for the competitor news chain
step="competitor"
prompts = load_prompts(
    directory=prompt_file_path,
    filename_pattern='news.yml',
    filters={"step": step}
)
prompts_index = prompts.index()

run_id = f"monday_briefing_{today.month}.{today.day}.{str(today.year)[-2:]}"

news_prompt = prompts_index[step]["news_research"]
news_prompt.run_id = run_id
parse_prompt = prompts_index[step]["news_parse"]
parse_prompt.run_id = run_id
parse_prompt.response_format=NewsInsightList
review_prompt = prompts_index[step]["news_review"]
review_prompt.run_id = run_id
review_prompt.response_format=NewsReviewList
summary_prompt = prompts_index[step]["summarize_news_topic"]
summary_prompt.run_id = run_id
summary_prompt.response_format=SummaryResponseList

INFO:prompt_chain.utils.loader:Loaded 1 prompt file(s)


INFO:prompt_chain.utils.loader:Loaded 5 prompts from ..\prompts


In [ ]:

def company_news_chain(company):
    #Search for news
    news_response = llm.run_prompt(
        prompt=news_prompt,
        variables={"company": company},
        context={"company": company}
    )
    news_response.save_to_file(base_dir=response_file_path, variable=company)

    #Parse news
    report=news_response.text
    citations=news_response.citations.as_markdown_list()
    parse_response = llm.run_prompt(
        prompt=parse_prompt,
        variables={
            "company": company,
            "report": report,
            "citations": citations
        },
        context={"company": company},
        citations=news_response.citations
    )
    parse_response.save_to_file(base_dir=response_file_path, variable=company)

    #Review news for relevance and likely date
    review_response = llm.run_prompt(
        prompt=review_prompt,
        variables={
            "company": company,
            "items": parse_response.text
        },
        context={"company": company},
        citations=news_response.citations
    )
    review_response.save_to_file(base_dir=response_file_path, variable=company)

for company in companies:
    company_news_chain(company)

In [6]:
from functools import reduce
from projects.monday_briefing.utils.pdf_exporter import generate_summary_pdf, generate_pdf_from_markdown

# 1. Load and combine reviews
review_lists = [
    NewsReviewList.from_dict(response.structured)
    for response in load_prompt_responses(
        directory=response_file_path,
        filename_pattern='news_review*04-25T*.json'
    )
]

combined = reduce(lambda a, b: a.join(b), review_lists) if review_lists else NewsReviewList(items=[])

# 2. filter to only those you want
summary_only = combined.filter_by_include(True)

# 3. Group the summaries by topic
organized_reviews = summary_only.organize_by_category(topics)

INFO:prompt_chain.utils.loader:Loaded 16 responses from ../outputs/responses


In [ ]:
for topic, review_list in organized_reviews.items():
    news_items = []
    for i in review_list.items:
        sources = '\n'.join([f"{s}" for s in i.sources])
        news_item = f"{i.summary}\nsources:{sources}"
        news_items.append(news_item)

    news_item_string = '\n\n'.join(news_items)

    #Parse news
    response = llm.run_prompt(
        prompt=summary_prompt,
        variables={
            "topic": topic,
            "news_items": news_item_string
        },
        context={"topic": topic}
    )
    response.save_to_file(base_dir=response_file_path, variable=topic, print_save=False)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [ ]:
from functools import reduce
from projects.monday_briefing.utils.pdf_exporter import generate_topic_summary_pdf

# 1. Load and combine reviews
summary_lists = [
    SummaryResponseList.from_dict(response.structured)
    for response in load_prompt_responses(
        directory=response_file_path,
        filename_pattern='news_review*04-25T*.json'
    )
]

combined = reduce(lambda a, b: a.join(b), summary_lists) if summary_lists else SummaryResponseList(items=[])


### Usage
```
Completion Tokens:   436
Prompt Tokens:       708
Total Tokens:        1,144
Estimated Cost:      $0.006130
```


### Response Metadata
| Field | Value |
|---|---|
| Model | gpt-4o |
| Temperature | 0.3 |
| Max Tokens | 2048 |
| Top-K | 40 |
| Top-P | 0.9 |
| Created | 2025-04-27 21:44:23+00:00 |


### Output
```
```json
{
  "topic": "Products",
  "summary": [
    "Keyston Brothers launched the final additions to their Podium Collection, enhancing their luxury automotive interior offerings with high-end leathers and fabrics[1][2].",
    "S&F Supplies introduced the HP Latex R530 All-in-One Printer, focusing on innovative and space-efficient printing solutions, and launched the Avery Dennison Auravate Decorative Window Film Series, featuring seven premium films for glass enhancements[3].",
    "Trivantage announced the Batyline® Sling Fabric Collection, offering two distinct collections from Serge Ferrari, which expands their performance fabric offerings and reinforces their market position in technical textiles[4][5].",
    "United Fabrics introduced 'Blocker,' a new product with a unique embossed texture mimicking sewn leather, and continues to focus on the 'Red Rocks' collection created in collaboration with Sunbrella, inspired by the American Southwest with 24 SKUs of performance fabrics in earth-toned colorways[6][7][8]."
  ],
  "sources": [
    "https://www.thehogring.com/2025/04/10/keyston-bros-debuts-final-podium-collection-leathers/",
    "https://keystonbros.com/blogs",
    "https://sfsupplies.com/news/",
    "https://www.textileworld.com/textile-world/nonwovens-technical-textiles/2025/04/trivantage-launches-batyline-sling-fabric-collection-two-distinct-collections-from-serge-ferrari/",
    "https://casualnewsnow.com/blog/2025/04/09/trivantage-launches-batyline-sling-fabric-collection/",
    "https://www.instagram.com/p/DI1x5XhTGxX/",
    "https://www.unitedfabrics.com",
    "https://www.textileworld.com/textile-world/2024/10/united-fabrics-and-sunbrella-look-to-the-southwest-for-new-red-rocks-collection/"
  ]
}
```
```